# Classes and objects

[http://docs.scala-lang.org/tutorials/tour/classes](http://docs.scala-lang.org/tutorials/tour/classes)

A simple class definition in Scala:

In [ ]:
import java.util.Calendar // importing a Java class

// A utility function to use in the class definition
def currentYear() = Calendar.getInstance.get(Calendar.YEAR)

class Person(name: String, birthYear: Int) {           // class name and default-constructor
    
    def this(name: String) = this(name, currentYear()) // alternative constructor
    
    def age(): Integer = {
        currentYear() - birthYear
    }
    
    def canVote(): Boolean = this.age() >= 18          // `this` keyword, like in Java
    
    override def toString(): String = s"$name, $age"   // override Object#toSTring
}

Remarks:
* We start by importing a Java class. Scala interoperates very well with Java.
* Like in java, the keyword `class` is used to define a new class.
* After the class name there is an argument list. These are the arguments to the default constructor for the class. The variables `name` and `birthYear` are available throughout the class definition.
* Alternative constructors are declared by defining a method named `this`. The first statement in an alternative constructor must call the default constructor.
* Like in Java, the keyword `this` is used to refer to the current object inside a method body.
* `override` is a keyword (instead of ann annotation) in Scala. The compiler will complain if you unintentionally override a method without using this keyword.
* My implementation of `toString` uses [string interpolation](http://docs.scala-lang.org/overviews/core/string-interpolation.html)

New instances of classes are created by using the `new` keyword.

In [ ]:
val alice = new Person("Alice", 1971)
val bob = new Person("Bob", 1985)
val carol = new Person("Carol")

In [ ]:
alice.age()
bob.toString()
carol.canVote()

In addition to defining general classes with the `class` keyword, Scala defines the `object` keyword for creating [singleton objects](http://docs.scala-lang.org/tutorials/tour/singleton-objects.html).

In [ ]:
class Widget(color: String) {
}

object Factory {
    def makeWidget(color: String): Widget = {
        println("clank clank")
        new Widget(color)
    }
}

In [ ]:
val widget = Factory.makeWidget("blue")

## Companion objects

Scala does not have a `static` keyword. In situations where you would define a static method in Java, it is typical to define a singleton object with the same name as your class. The singleton is referred to as the "companion object" for the class. For example, if we want to define additional constructors for `Person` you might add them as factory methods to a companion object.

In [ ]:
object Person {
    // Construct a person from a delimited string
    def fromDelimitedString(string: String, delimiter: String = ","): Person = {
        val fields = string.split(delimiter)
        val name = fields(0)
        val birthYear = fields(1).toInt
        new Person(name, birthYear)
    }
}

In [ ]:
val david = Person.fromDelimitedString("David,1999")

## Optional parentheses for methods with no arguments

If a method doesn't take any arguments, you can optionally declare it without parentheses. Methods defined this way must be invoked without parenthesis.

In [ ]:
object Factory {
    val POLLUTION_PER_WIDGET = 9
    var numberOfWidgetsMade = 0
    
    def makeWidget(color: String): Widget = {
        println("clank clank")
        numberOfWidgetsMade += 1
        new Widget(color)
    }
    
    def totalPollution = numberOfWidgetsMade * POLLUTION_PER_WIDGET
}

In [ ]:
Factory.makeWidget("puce")
Factory.makeWidget("vermillion")
Factory.totalPollution

Calling `totalPollution` with parentheses will raise an exception.

In [ ]:
Factory.totalPollution()

## Getters and setters

By default, the arguments to the default constructor (i.e., the parameters list that follows the class name) are not accessible outside the class definition. Declaring arguments with `val` and `var` keywords will create getters and setters for them.

Also note that for classes that don't define any methods or fields outside of the default constructor, curly braces are not required.

In [ ]:
class Widget(val color: String, var size: Int)

In [ ]:
val w = new Widget("blue", 10)
w.color
w.size = 11
w.size

There is also a special syntax to create getters and setters that look like property access and assignment. The syntax builds on the "no parentheses" syntax above.

In [ ]:
class Sprocket(var widthInCentimeters: Int) {
    def widthInMeters = widthInCentimeters / 100.0  // getter
    
    def widthInMeters_=(meters: Double) = {         // setter
        widthInCentimeters = (meters * 100).toInt
    }
}

In [ ]:
val sprocket = new Sprocket(100)
sprocket.widthInMeters = 0.89
sprocket.widthInCentimeters

## The `apply` method

If your class defines a method named `apply`, then instances of the class can be treated like functions.

In [ ]:
class Adder(n: Int) {
    def apply(x: Int) = x + n
}

In [ ]:
val add7 = new Adder(7)

// function syntax delegates to the `apply` method
add7(2)
add7(11)

We saw an example of this when we were working with Scala's collection library. Scala collections generally have a companion object that implements the `apply` method to be a factory method for the class. This is how `List(1, 2, 3)` constructs a new `List` instance even though we didn't use the `new` keyword.

In [ ]:
class Container[T](val stuff: List[T])    // generic class with type parameter T


object Container {
    def apply[T](args: T*) = {
        new Container[T](args.toList)
    }
}

// Construct a new Container using the `apply` method in the companion object
val container = Container(1, 2, 3)
container.stuff

## Case classes

Scala combines several of these features in a concept called case classes. `case` is a Scala keyword that lets you define classes that behave like [values](https://en.wikipedia.org/wiki/Value_object), meaning that two instances of the class should be treated the same if their corresponding fields are all equal. When you define a case class, Scala provides the following functionality:

* Getters are defined for the default argument. By default they are final, but you can change this by supplying the `var` keyword.
* `==`, `hashCode`, and `toString` are implemented. Two instances of the class are equal if their corresponding fields are equal.
* The class name can be used as a constructor for the class without the `new` keyword.
* The class name can be used for pattern matching in case statements.

We'll come back to the last point when we cover `match` and `case` in another notebook. Let's look at an example.

In [ ]:
case class Pair[T1, T2](x: T1, y: T2)

In [ ]:
val p1 = Pair(1.0, 2.0) // Using the class name as a constructor
val p2 = Pair(1.0, 2.0)
val p3 = Pair(3.0, 4.0)

p1 == p2                // equality determined by instance fields
p1 == p3

p1.hashCode             // hashCode is implemented
p2.hashCode
p3.hashCode

In [ ]:
p1.x = 11 // compilation error, x is final

Case classes are a great way to eliminate boilerplate when defining classes whose purpose is to act as containers for their fields. You can add methods to a case class just like a regular class.

In [ ]:
case class ComplexNumber(re: Double, im: Double) {
    def modulus = Math.sqrt(Math.pow(re, 2) + Math.pow(im, 2))
}

In [ ]:
val z = ComplexNumber(1, 1)
z.modulus

## Inheritance and traits